# Face Recognition using CNN and LBPHFaceRecognizer

Importing libraries

In [37]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.utils import to_categorical
from keras.models import load_model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
import itertools

import os
import cv2
import numpy as np
from PIL import Image
import pickle
from numpy import asarray

# Making train and test datasets

In [38]:
face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')

In [39]:
image_dir = 'D:\Python Projects\Face-Recognizer\src\\train_images'
current_id = 0
label_ids = {}
y_train = []
x_train = []
i = 1
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg") or file.endswith("jpeg") or file.endswith("JPG"):
            path = os.path.join(root, file)
            label = os.path.basename(root).replace(" ", "-").lower()
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
                i = 1
            id_ = label_ids[label]
            gray = Image.open(path).convert("L") #grayscale
            gray = np.array(gray, "uint8")
            faces = face_cascade.detectMultiScale(gray)
            for (x, y, w, h) in faces:
                roi = gray[y : y + h, x : x + w]
                size = (64, 64)
                pil_image = Image.fromarray(roi.astype('uint8'), 'L')
                final_image = pil_image.resize(size, Image.ANTIALIAS)
                image_array = np.array(final_image, "uint8")
                i = i + 1
                x_train.append(image_array)
                y_train.append(id_)

In [40]:
len(x_train), len(y_train)

(180, 180)

In [41]:
image_dir = 'D:\Python Projects\Face-Recognizer\src\\test_images'
current_id = 0
label_ids = {}
y_test = []
x_test = []
i = 1
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg") or file.endswith("jpeg") or file.endswith("JPG"):
            path = os.path.join(root, file)
            label = os.path.basename(root).replace(" ", "-").lower()
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
                i = 1
            id_ = label_ids[label]
            gray = Image.open(path).convert("L") #grayscale
            gray = np.array(gray, "uint8")
            faces = face_cascade.detectMultiScale(gray)
            for (x, y, w, h) in faces:
                roi = gray[y : y + h, x : x + w]
                size = (64, 64)
                pil_image = Image.fromarray(roi.astype('uint8'), 'L')
                final_image = pil_image.resize(size, Image.ANTIALIAS)
                image_array = np.array(final_image, "uint8")
                i = i + 1
                x_test.append(image_array)
                y_test.append(id_)

In [42]:
len(x_test), len(y_test)

(60, 60)

In [43]:
label_ids

{'bharat-choithani': 0,
 'emilia-clarke': 1,
 'kit-harington': 2,
 'lena-headey': 3,
 'nikolaj-coster-waldau': 4,
 'peter-dinklage': 5}

Normalization on the dataset

In [44]:
x_train = [x/255 for x in x_train]
x_train = np.array(x_train, dtype='float32')
x_test = [x/255 for x in x_test]
x_test = np.array(x_test, dtype='float32')

# LBPHFaceRecognizer

Train the model

In [45]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(x_train, np.array(y_train))

Predictions

In [46]:
y_pred = []
for i in x_test:
    id_, conf = recognizer.predict(i)
    y_pred.append(id_)

accuracy = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        accuracy += 1

In [47]:
print("Accuracy = ", (accuracy/len(y_pred)) * 100)

Accuracy =  86.66666666666667


In [48]:
recognizer.save("LBPHFaceRecognizer.yml")

# Keras CNN

Pre processing for CNN model

In [49]:
x_train = [x[..., np.newaxis] for x in x_train]
x_train = np.array(x_train, dtype='float32')
x_test = [x[..., np.newaxis] for x in x_test]
x_test = np.array(x_test, dtype='float32')

In [50]:
x_train.shape, x_test.shape

((180, 64, 64, 1), (60, 64, 64, 1))

Build CNN model:
 CNN have 3 main layer: 
 * 1-Convolotional layer 
 * 2- pooling layer  
 * 3- fully connected layer

In [51]:
input_shape = (64, 64, 1)

In [52]:
cnn_model= Sequential([
    Conv2D(filters=16, kernel_size=7, activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=16, kernel_size=5, activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(2024, activation='relu'),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(set(y_train)), activation='softmax')  
])

cnn_model.compile(
    loss='sparse_categorical_crossentropy',#'categorical_crossentropy',
    optimizer=Adam(lr=0.0001),
    metrics=['accuracy']
)

Show the model's parameters.

In [53]:
cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 58, 58, 16)        800       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 29, 29, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 25, 25, 16)        6416      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 2024)              4665320   
_________________________________________________________________
dropout_6 (Dropout)          (None, 2024)             

Train the model


In [54]:
history=cnn_model.fit(
    np.array(x_train), np.array(y_train), batch_size=512,
    epochs=200, verbose=2,
)

Epoch 1/200
1/1 - 0s - loss: 1.7946 - accuracy: 0.1556
Epoch 2/200
1/1 - 0s - loss: 1.8060 - accuracy: 0.1556
Epoch 3/200
1/1 - 0s - loss: 1.7714 - accuracy: 0.2167
Epoch 4/200
1/1 - 0s - loss: 1.7920 - accuracy: 0.1778
Epoch 5/200
1/1 - 0s - loss: 1.7746 - accuracy: 0.2556
Epoch 6/200
1/1 - 0s - loss: 1.7806 - accuracy: 0.1500
Epoch 7/200
1/1 - 0s - loss: 1.7727 - accuracy: 0.1944
Epoch 8/200
1/1 - 0s - loss: 1.7445 - accuracy: 0.2722
Epoch 9/200
1/1 - 0s - loss: 1.7426 - accuracy: 0.2611
Epoch 10/200
1/1 - 0s - loss: 1.7405 - accuracy: 0.2722
Epoch 11/200
1/1 - 0s - loss: 1.7223 - accuracy: 0.3111
Epoch 12/200
1/1 - 0s - loss: 1.7453 - accuracy: 0.2667
Epoch 13/200
1/1 - 0s - loss: 1.7255 - accuracy: 0.2889
Epoch 14/200
1/1 - 0s - loss: 1.7111 - accuracy: 0.3056
Epoch 15/200
1/1 - 0s - loss: 1.7062 - accuracy: 0.2889
Epoch 16/200
1/1 - 0s - loss: 1.7021 - accuracy: 0.3222
Epoch 17/200
1/1 - 0s - loss: 1.7142 - accuracy: 0.2889
Epoch 18/200
1/1 - 0s - loss: 1.6902 - accuracy: 0.3167
E

Epoch 147/200
1/1 - 0s - loss: 0.2179 - accuracy: 0.9389
Epoch 148/200
1/1 - 0s - loss: 0.2039 - accuracy: 0.9389
Epoch 149/200
1/1 - 0s - loss: 0.1988 - accuracy: 0.9389
Epoch 150/200
1/1 - 0s - loss: 0.2073 - accuracy: 0.9500
Epoch 151/200
1/1 - 0s - loss: 0.1977 - accuracy: 0.9444
Epoch 152/200
1/1 - 0s - loss: 0.1684 - accuracy: 0.9611
Epoch 153/200
1/1 - 0s - loss: 0.1837 - accuracy: 0.9444
Epoch 154/200
1/1 - 0s - loss: 0.1662 - accuracy: 0.9611
Epoch 155/200
1/1 - 0s - loss: 0.1499 - accuracy: 0.9667
Epoch 156/200
1/1 - 0s - loss: 0.1687 - accuracy: 0.9389
Epoch 157/200
1/1 - 0s - loss: 0.1561 - accuracy: 0.9667
Epoch 158/200
1/1 - 0s - loss: 0.1669 - accuracy: 0.9556
Epoch 159/200
1/1 - 0s - loss: 0.1515 - accuracy: 0.9444
Epoch 160/200
1/1 - 0s - loss: 0.1544 - accuracy: 0.9556
Epoch 161/200
1/1 - 0s - loss: 0.1455 - accuracy: 0.9833
Epoch 162/200
1/1 - 0s - loss: 0.1436 - accuracy: 0.9611
Epoch 163/200
1/1 - 0s - loss: 0.1175 - accuracy: 0.9778
Epoch 164/200
1/1 - 0s - loss: 

Evaluate the test data


In [55]:
scor = cnn_model.evaluate(np.array(x_test), np.array(y_test), verbose=0)

print('test los {:.4f}'.format(scor[0]))
print('test acc {:.4f}'.format(scor[1]*100))

test los 0.4612
test acc 88.3333


In [56]:
cnn_model.save('my_model.h5')

In [57]:
cnn_model = load_model('my_model.h5')

# Real Time Face Detection and Recognition

In [59]:
face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
labels = {v : k for k, v in label_ids.items()}

cap = cv2.VideoCapture(0)

while(True):
    #Capture frame-by-frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.5, minNeighbors = 5)
    gray = np.array(gray, "uint8")
    
    for (x, y, w, h) in faces:
        
        roi_gray = gray[y : y + h, x : x + w]
        roi_color = frame[y : y + h, x : x + w]
             
        roi_gray = Image.fromarray(roi_gray.astype('uint8'), 'L')
        size = (64, 64)
        roi_gray = roi_gray.resize(size, Image.ANTIALIAS)
        roi_gray = asarray(roi_gray)
        
        roi_gray = np.array(roi_gray, dtype='float32')/255
        #Recognize face using LBPHFaceRecognizer
        #id_, conf = recognizer.predict(roi_gray)
        #print(labels[id_])
        #name = labels[id_]
        
        roi_gray = roi_gray[np.newaxis, ..., np.newaxis]
        #Recognize face using Keras CNN
        id_ = cnn_model.predict_classes(roi_gray)
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        name = labels[id_[0]]
        
        color = (255, 255, 255)
        stroke = 2
        cv2.putText(frame, name, (x, y), font, 1, color, stroke, cv2.LINE_AA)
        
        color = (255, 0, 0) #BGR 0-255
        stroke = 2
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, stroke)
        
    #Display the resulting frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()